In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Attempt to import the appropriate QASM simulator backend
try:
    from qiskit.providers.basicaer import BasicAer
    backend = BasicAer.get_backend('qasm_simulator')
except ImportError:
    try:
        from qiskit.providers.aer import QasmSimulator
        backend = QasmSimulator()
    except ImportError:
        raise ImportError(
            "No QASM simulator found. Please install qiskit-terra or qiskit-aer."
        )

from qiskit.utils import QuantumInstance
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.algorithms.optimizers import SPSA
from qiskit.algorithms import VQE

def setup_lih_hamiltonian(bond_length: float):
    """Generate LiH Hamiltonian with STO-3G and Jordan-Wigner mapping."""
    driver = PySCFDriver(atom=f"Li 0 0 0; H 0 0 {bond_length}", basis='sto3g')
    problem = ElectronicStructureProblem(driver)
    second_q_ops = problem.second_q_ops()
    electronic_hamiltonian = second_q_ops[0]
    converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=False)
    qubit_op = converter.convert(
        electronic_hamiltonian, num_particles=problem.num_particles
    )
    return qubit_op, problem, converter

def build_uccsd_ansatz(problem, converter):
    """Construct UCCSD ansatz on HF reference."""
    num_particles = problem.num_particles
    num_spin_orbitals = problem.num_spin_orbitals
    initial_state = HartreeFock(
        num_spin_orbitals, num_particles, qubit_converter=converter
    )
    ansatz = UCCSD(
        qubit_converter=converter,
        num_particles=num_particles,
        num_spin_orbitals=num_spin_orbitals,
        initial_state=initial_state
    )
    return ansatz

def estimate_energy_z_sampling(counts, qubit_op):
    """Estimate energy from Z-basis sample counts."""
    total_shots = sum(counts.values())
    energy = 0.0
    for bitstring, freq in counts.items():
        p_z = freq / total_shots
        E_z = 0.0
        for pauli, coeff in qubit_op.paulis.to_list():
            # Only diagonal (I/Z) terms contribute
            diag = True
            parity = 1
            for idx, op in pauli:
                if op == 'Z':
                    if bitstring[-1-idx] == '1':
                        parity *= -1
                elif op != 'I':
                    diag = False
                    break
            if diag:
                E_z += coeff * parity
        energy += p_z * E_z
    return energy

def vqe_convergence(bond_length, shots=1000, maxiter=50):
    """Run VQE with both QWC and Z-sampling, return convergence histories."""
    qubit_op, problem, converter = setup_lih_hamiltonian(bond_length)
    ansatz = build_uccsd_ansatz(problem, converter)
    
    qi = QuantumInstance(backend, shots=shots)

    hist_qwc = []
    hist_z = []

    # QWC via VQE callback
    def cb_qwc(eval_count, params, mean, std):
        hist_qwc.append(mean)
    optimizer = SPSA(maxiter=maxiter)
    vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi, callback=cb_qwc)
    _ = vqe.compute_minimum_eigenvalue(qubit_op)

    # Z-sampling with SPSA.optimize callback
    def objective_z(theta):
        qc = ansatz.bind_parameters(theta)
        result = backend.run(qc, shots=shots).result()
        counts = result.get_counts()
        return estimate_energy_z_sampling(counts, qubit_op)

    def cb_z(eval_count, params, mean, std):
        hist_z.append(mean)
    optimizer_z = SPSA(maxiter=maxiter, callback=cb_z)
    initial_point = np.zeros(ansatz.num_parameters)
    _ = optimizer_z.optimize(
        num_vars=ansatz.num_parameters,
        objective_function=objective_z,
        initial_point=initial_point
    )

    return hist_qwc, hist_z

def plot_convergence(hist_qwc, hist_z, hci_energy):
    """Plot convergence curves against HCI reference."""
    iters = list(range(len(hist_qwc)))
    plt.plot(iters, hist_qwc, label='QWC VQE')
    plt.plot(iters, hist_z, label='Z-sampling VQE')
    plt.axhline(hci_energy, label='HCI Reference', linestyle='--')
    plt.xlabel('SPSA Iteration')
    plt.ylabel('Energy (Hartree)')
    plt.title('VQE Convergence Comparison')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Example usage
if __name__ == "__main__":
    bond_length = 1.596  # equilibrium Li-H distance in Å
    hci_ref = -7.8823    # HCI reference energy
    maxiter = 30
    shots = 1000

    hist_qwc, hist_z = vqe_convergence(
        bond_length, shots=shots, maxiter=maxiter
    )
    plot_convergence(hist_qwc, hist_z, hci_ref)


ImportError: No QASM simulator found. Please install qiskit-terra or qiskit-aer.